In [1]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
import math

from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import r2_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split,cross_val_score, KFold,validation_curve,GridSearchCV
from sklearn.preprocessing import LabelEncoder

import joblib
from datetime import datetime


import ModelFunctions

C:\Users\nining\AppData\Local\Temp\ipykernel_10268\2844815712.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data = pd.read_csv("data_pre_process.csv")

Xtrain_fso, Xval_fso, Xtest_fso, Ytrain_fso, Yval_fso, Ytest_fso = \
    ModelFunctions.split_data1(data, "FSO_Att",drop_column = ["Frequency",'RFL_Att']) # Frequency doesn't relate to FSO_Att

Xtrain_rfl, Xval_rfl, Xtest_rfl, Ytrain_rfl, Yval_rfl, Ytest_rfl = \
    ModelFunctions.split_data1(data, "RFL_Att",drop_column = ['FSO_Att'])





#######  FSO_Att Data Set  ######
FSO_Att Training set feature count: (63965, 24)
FSO_Att Validation set feature count: (27414, 24)
FSO_Att Training set label count: (63965,)
FSO_Att Validation set label count: (27414,)
#######  RFL_Att Data Set  ######
RFL_Att Training set feature count: (63965, 25)
RFL_Att Validation set feature count: (27414, 25)
RFL_Att Training set label count: (63965,)
RFL_Att Validation set label count: (27414,)


In [4]:
Xtrain_rfl.columns

Index(['AbsoluteHumidity', 'AbsoluteHumidityMax', 'AbsoluteHumidityMin',
       'Distance', 'Frequency', 'Particulate', 'ParticulateMax',
       'ParticulateMin', 'RainIntensity', 'RainIntensityMax',
       'RainIntensityMin', 'RelativeHumidity', 'SYNOPCode', 'Temperature',
       'TemperatureDifference', 'TemperatureMax', 'TemperatureMin', 'Time',
       'Visibility', 'VisibilityMax', 'VisibilityMin', 'WindDirection',
       'WindSpeed', 'WindSpeedMax', 'WindSpeedMin'],
      dtype='object')

In [5]:
set(data['SYNOPCode'])

{0, 3, 4, 5, 6, 7, 8}

In [ ]:

            # sorted_features, sorted_importance = \
            #     sort_features(model,Xtrain,channel = "FSO",figure = False)


In [3]:
# define SYNOPCode value
synop_codes = [0, 3, 4, 5, 6, 7,8]
synop_codes = [9]

# create empty list to store result
performance_scores_list = []
df_list = []
rfl_rf_model = RandomForestRegressor(n_estimators=130, max_depth=25, \
                                        min_samples_leaf=1, random_state=25,\
                                        min_samples_split=2, oob_score=True, n_jobs=6)
fso_rf_model = RandomForestRegressor(n_estimators = 130, max_depth=30, \
                                     min_samples_leaf = 1, random_state = 25,\
                                     min_samples_split =2, oob_score=True, n_jobs=6 )

for code in synop_codes:
    # Split the dataset into training and validation sets based on the values in the SYNOPCode column
    Xtrain_rfl_s, Ytrain_rfl_s = ModelFunctions.filter_data_by_SYNOPCode(Xtrain_rfl, Ytrain_rfl, code)
    Xval_rfl_s, Yval_rfl_s = ModelFunctions.filter_data_by_SYNOPCode(Xval_rfl, Yval_rfl, code)
    Xtrain_fso_s, Ytrain_fso_s = ModelFunctions.filter_data_by_SYNOPCode(Xtrain_fso, Ytrain_fso, code)
    Xval_fso_s, Yval_fso_s = ModelFunctions.filter_data_by_SYNOPCode(Xval_fso, Yval_fso, code)
    # store metrics to DataFrame
    performance_scores = ModelFunctions.wrapper_method (rfl_rf_model,Xtrain_rfl_s,Ytrain_rfl_s,Xval_rfl_s,Yval_rfl_s,channel = "RFL")
    important_columns = ["remove_feature", "train_rmse", "train_r2","val_rmse","val_r2", "oob_r2","next_remove","importance_score"]
    df = pd.DataFrame(performance_scores, columns=important_columns)
    # write to csv
    file_name = f"rfl_synopcode{code}_130-25-1-2_feature_importance_wrapple.csv"
    df.to_csv(file_name, index=False)

    performance_scores = ModelFunctions.wrapper_method (fso_rf_model,Xtrain_fso_s,Ytrain_fso_s,Xval_fso_s,Yval_fso_s,channel = "FSO")
    df = pd.DataFrame(performance_scores, columns=important_columns)
    file_name = f"FSO_synopcode{code}_130-30-1-2_feature_importance_wrapple.csv"
    df.to_csv(file_name, index=False)



c:\Users\nining\.conda\envs\bigdata\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\nining\.conda\envs\bigdata\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Training RMSE: 0.19801796694680984
Validation RMSE: 0.5060067721023563
Train R²: 0.9966765675677622
Validation R²: 0.9784647932420589
OOB R²: 0.9761913367317926
1 WindDirection ['WindSpeedMin', 'WindSpeed', 'WindSpeedMax', 'ParticulateMin', 'TemperatureMin', 'TemperatureMax', 'Particulate', 'VisibilityMin', 'AbsoluteHumidityMax', 'TemperatureDifference', 'VisibilityMax', 'AbsoluteHumidityMin', 'Time', 'Visibility', 'Frequency', 'ParticulateMax', 'RelativeHumidity', 'SYNOPCode', 'Temperature', 'RainIntensityMin', 'RainIntensityMax', 'RainIntensity', 'Distance', 'AbsoluteHumidity']
%%%%%%%%%%%%%%%%%%%%%%%%%%%%


c:\Users\nining\.conda\envs\bigdata\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\nining\.conda\envs\bigdata\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Training RMSE: 0.19667473185131315
Validation RMSE: 0.5018527256446286
Train R²: 0.9967215029853677
Validation R²: 0.9788169270550378
OOB R²: 0.9764531771826958
2 WindSpeed ['WindSpeedMin', 'WindSpeedMax', 'ParticulateMin', 'TemperatureMin', 'TemperatureMax', 'Particulate', 'VisibilityMin', 'AbsoluteHumidityMax', 'TemperatureDifference', 'VisibilityMax', 'AbsoluteHumidityMin', 'Time', 'Visibility', 'Frequency', 'ParticulateMax', 'RelativeHumidity', 'SYNOPCode', 'Temperature', 'RainIntensityMin', 'RainIntensityMax', 'RainIntensity', 'Distance', 'AbsoluteHumidity']
%%%%%%%%%%%%%%%%%%%%%%%%%%%%


c:\Users\nining\.conda\envs\bigdata\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\nining\.conda\envs\bigdata\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


...

c:\Users\nining\.conda\envs\bigdata\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\nining\.conda\envs\bigdata\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Training RMSE: 0.6907274458458199
Validation RMSE: 1.8014882200576052
Train R²: 0.9685816068862161
Validation R²: 0.7888030783334661
OOB R²: 0.7755951626819815
22 Temperature ['Visibility', 'Distance']
%%%%%%%%%%%%%%%%%%%%%%%%%%%%


c:\Users\nining\.conda\envs\bigdata\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\nining\.conda\envs\bigdata\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Training RMSE: 1.0967260775193117
Validation RMSE: 2.5753216135191894
Train R²: 0.9207923946371652
Validation R²: 0.5683937987275629
OOB R²: 0.5543332531038505
23 Visibility ['Distance']
%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Training RMSE: 2.4163844830684913
Validation RMSE: 3.3505369178472133
Train R²: 0.6154944436511963
Validation R²: 0.2694439432186043
OOB R²: 0.2683462752536413
24 Distance []
%%%%%%%%%%%%%%%%%%%%%%%%%%%%


c:\Users\nining\.conda\envs\bigdata\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\nining\.conda\envs\bigdata\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
